In [13]:
from pymongo import MongoClient, CursorType
from pymongo.cursor import Cursor
from sshtunnel import SSHTunnelForwarder
import asyncio
import itertools
import numpy as np

import numpy as np
import logging
logger = logging.getLogger(__name__)
import asyncio
from angie.libs.singleton import singleton
from angie.libs.mongo_utils import iterate_by_chunks
from angie.datacontracts.msg_similarity import *
from angie.datacontracts.msg import Message
from angie.libs.tensorflow_utils import sim_exact
from angie.service.app_config import AppConfig
from angie.datacontracts.http.msg_filter import *
from angie.datacontracts.http.msg_representation_response import *
from angie.datacontracts.msg_rerpresentation import MsgEmbRepresentation
from angie.service.managers.genie_msg_manager import GenieMsgManager
from angie.datacontracts.msg_similarity import MsgSimilarity
from angie.libs.embed.msg_representation import MsgRPRKey

In [2]:
# MONGO_HOST = 'ec2-52-23-187-115.compute-1.amazonaws.com'
# MONGO_DB = "marketpulzz"
# server = SSHTunnelForwarder(
#     MONGO_HOST,
#     ssh_username='ubuntu',
#     ssh_pkey="/Users/sasha/.ssh/mmuze.pem",
# #     ssh_private_key_password="secret",
#     remote_bind_address=('127.0.0.1', 27017),
#     local_bind_address=('127.0.0.1', 63329),
#     set_keepalive = 5,
# )

# server.start()
# client = MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
# db = client[MONGO_DB]

In [3]:
MONGO_DB = "marketpulzz"
client = MongoClient('127.0.0.1', 27017)

# read_preference = ReadPreference.PRIMARY
# mongo_hosts : ["ec2-52-23-187-115.compute-1.amazonaws.com:27017", "ec2-52-90-96-8.compute-1.amazonaws.com:27017"]
# client = MongoClient(mongo_hosts, read_preference = read_preference)

db = client[MONGO_DB]

In [4]:
def iterate_by_chunks(collection, chunksize=1, batchsize=0, start_from=0, query={}, projection={}):
    '''read mongo collection by chunks'''
    chunks = range(start_from, collection.find(query).count(), int(chunksize))
    num_chunks = len(chunks)
    for i in range(1,num_chunks+1):
        if i < num_chunks:
            yield collection.find(query, projection=projection)[chunks[i-1]:chunks[i]]
        else:
            yield collection.find(query, projection=projection)[chunks[i-1]:chunks.stop]

In [5]:
async def fetch_by_chunks(cursor: Cursor, chunk_size: 100):
    current_chunk = []
    for doc in cursor:
        current_chunk.append(doc)
        if len(current_chunk) == chunksize:
            yield current_chunk
            current_chunk = []
    yield current_chunk  

In [14]:
async def get_top_k_similar_msgs_for_query_vector_async(query_vector: np.ndarray, K: int, rpr_key: MsgRPRKey, sim_measure: SimilarityMeasure, messages_filter: MsgFilter=None, chunk_size=100) -> List[MsgSimilarity]:

    _chunk_size = max(K, chunk_size)
    msg_embs_iter = iterate_by_chunks(db.genie_conversation_messages_embs, chunksize=_chunk_size, start_from=0,
                                        query={'rpr_key':rpr_key}, projection={ 'msg_vector':1, 'genie_conversation_message_id':1 })
        
    def get_top_k_similar(msg_sims: List[MsgSimilarity], K: int):        
        k_similar = list(sorted(msg_sims, key=lambda ms: ms.similarity_score ,reverse=True))
        top_k_similar = k_similar[:K]  
        return top_k_similar
    
    msgs_queue = asyncio.Queue()
    msg_sims_res = []
    
    async def enque_msgs(items: List) -> None:
        chunk_n = chunk_n + 1
        logging.info(f'chunk #: {chunk_n}, chunk_len: {len(items)}')
        await msgs_queue.put(items)

    
    async def calc_sim():        
        while True:
            msgs_embs_docs = await msgs_queue.get()        
            msg_vectors = list(map(lambda me: me['msg_vector'], msgs_embs_docs))
            sims_res = sim_exact([query_vector], np.array(msg_vectors))        
            sim_scores = {
                    SimilarityMeasure.ANGDIST: sims_res[0],
                    SimilarityMeasure.COSSIM: sims_res[1]
                }.get(sim_measure)
            msg_sims = [MsgSimilarity(str(msg['genie_conversation_message_id']), float(sim_score[0])) for msg, sim_score in zip(msgs_embs_docs, sim_scores)]
            top_k_similar = get_top_k_similar(msg_sims, K)            
            msg_sims_res.append(top_k_similar)    
#             yield msg_sims
            msgs_queue.task_done()
   
    top_k_similar = []
    chunk_n = 0      
    try:
        sim_calculation_batch_size = 1000
        num_sim_calculators = 10
        producers = [asyncio.create_task(enque_msgs(fetch_by_chunks(msg_embs_cursor, chunk_size=sim_calculation_batch_size))) for msg_embs_cursor in msg_embs_iter]    
        consumers = [asyncio.create_task(calc_sim) for n in range(num_sim_calculators)]
        await asyncio.gather(*producers)
        await msgs_queue.join()
        for c in consumers:
            c.cancel()
    
        msg_sims_res_flat = list(itertools.chain(*msg_sims_res))
        top_k_similar = get_top_k_similar(msg_sims_res_flat, K)
        
        k_similar = list(sorted(msg_sims_res, key=lambda ms: ms.similarity_score ,reverse=True))
        top_k_similar = k_similar[:K]
    
        
    except Exception as e:
        logger.exception(e)
        raise e
  
    return top_k_similar
   
    


In [15]:
asyncio.run(get_top_k_similar_msgs_for_query_vector_async)

AttributeError: module 'asyncio' has no attribute 'run'

In [17]:
asyncio.run

AttributeError: module 'asyncio' has no attribute 'run'

In [18]:
!python --version


Python 3.6.8 :: Anaconda, Inc.
